In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from scipy.stats import spearmanr # Para a estabilidade SHAP
from sklearn.model_selection import train_test_split
import shap # Certifique-se de que a biblioteca SHAP está instalada
# from aif360.metrics import ClassificationMetric # Ignorando por enquanto, usando cálculo manual de EOD

# Define o nome da coluna sensível para uso consistente
SENSITIVE_FEATURE_NAME = 'Q'

ModuleNotFoundError: No module named 'shap'

In [ ]:
import pandas as pd

def load_and_combine_data(directory_path):
    X_train = pd.read_csv(f"{directory_path}/X_train.csv")
    y_train = pd.read_csv(f"{directory_path}/y_train.csv", header=None, names=['Y'])
    train_data = pd.concat([X_train, y_train], axis=1)
    
    X_test = pd.read_csv(f"{directory_path}/X_test.csv")
    y_test = pd.read_csv(f"{directory_path}/y_test.csv", header=None, names=['Y'])
    test_data = pd.concat([X_test, y_test], axis=1)
    
    # Assumimos que a primeira coluna de X_train é a variável sensível Q
    # (O Bias on Demand geralmente a nomeia como 'Q' ou um proxy)
    # Precisamos confirmar o nome da coluna Q quando gerarmos os dados
    
    return train_data, test_data

In [ ]:
def run_experiment(train_data, test_data, baseline_feature_ranking=None):
    """Treina o modelo, calcula métricas e estabilidade SHAP."""
    
    results = {}
    
    # Separar Features (X) e Target (y)
    X_train = train_data.drop('Y', axis=1)
    y_train = train_data['Y']
    X_test = test_data.drop('Y', axis=1)
    y_test = test_data['Y']

    # --- 1. Model Training ---
    model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # --- 2. Robustness Metrics ---
    results['Overall_Accuracy'] = accuracy_score(y_test, y_pred)
    results['Overall_F1_Score'] = f1_score(y_test, y_pred)
    results['AUROC'] = roc_auc_score(y_test, y_proba)
    
    # --- 3. Fairness Metrics (Equal Opportunity Difference - EOD) ---
    
    # Identificar a coluna Q e os grupos
    Q_test = X_test[SENSITIVE_FEATURE_NAME]
    
    # Cálculo do True Positive Rate (TPR)
    # TPR = Recall = TP / (TP + FN) = Proporção de verdadeiros positivos
    
    # Grupo Privilegiado (Q=0)
    TP_P = ((y_test == 1) & (Q_test == 0) & (y_pred == 1)).sum()
    P_pos = ((y_test == 1) & (Q_test == 0)).sum()
    TPR_P = TP_P / P_pos if P_pos > 0 else 0
    
    # Grupo Desprivilegiado (Q=1)
    TP_U = ((y_test == 1) & (Q_test == 1) & (y_pred == 1)).sum()
    U_pos = ((y_test == 1) & (Q_test == 1)).sum()
    TPR_U = TP_U / U_pos if U_pos > 0 else 0

    results['Equal_Opportunity_Difference'] = TPR_P - TPR_U
    results['Disparate_Impact_Ratio'] = (y_pred[Q_test == 1].mean() / 
                                        y_pred[Q_test == 0].mean()) if y_pred[Q_test == 0].mean() > 0 else 0

    # --- 4. Explainability Metrics (SHAP Stability) ---
    explainer = shap.Explainer(model, X_train)
    # Calcule os valores SHAP apenas para uma amostra para velocidade
    shap_values = explainer(X_test.sample(n=min(500, len(X_test)), random_state=42)) 
    
    # Média Absoluta SHAP por feature (Feature Importance)
    mean_abs_shap = pd.Series(np.abs(shap_values.values).mean(axis=0), index=X_test.columns)
    
    # Armazenar o ranking atual para comparação (ordenado pelos valores)
    current_feature_ranking = mean_abs_shap.sort_values(ascending=False).rank(method='first')

    results['Feature_Importance_Ranking'] = current_feature_ranking.to_dict()

    if baseline_feature_ranking is not None:
        # Calcular a correlação de posto de Spearman contra a linha de base
        # Usamos apenas as features comuns e na ordem correta
        
        # Converter para Series para garantir a ordem das features
        I_base = pd.Series(baseline_feature_ranking)
        I_current = current_feature_ranking
        
        # Alinhar os índices (nomes das features)
        common_features = I_base.index.intersection(I_current.index)
        
        # Spearman Rank Correlation
        rho, _ = spearmanr(I_base.loc[common_features].values, 
                           I_current.loc[common_features].values)
        results['SHAP_Rank_Stability'] = rho
    else:
        results['SHAP_Rank_Stability'] = 1.0 # Linha de base é 100% estável contra si mesma

    return results

In [ ]:
# Lista de diretórios de experimentos (substitua pelos seus caminhos reais)
experiment_directories = [
    '/path/to/baseline',
    # Bias Series (l_q)
    '/path/to/bias_lq_0.5', 
    '/path/to/bias_lq_0.7',
    # Noise Series (sy)
    '/path/to/noise_sy_0.1',
    '/path/to/noise_sy_0.2',
    # Imbalance Series (p_u)
    '/path/to/imbalance_pu_0.6',
    '/path/to/imbalance_pu_0.2'
]

master_results_df = pd.DataFrame()
baseline_ranking = None

print("--- Iniciando Loop de Auditoria ---")

for i, dir_path in enumerate(experiment_directories):
    print(f"\nProcessando: {dir_path.split('/')[-1]}")
    
    train_data, test_data = load_and_combine_data(dir_path)
    
    if train_data is None:
        continue # Pular se houver erro de carregamento

    # Determinar se é a linha de base
    is_baseline = (i == 0)

    # Executar o experimento
    current_results = run_experiment(train_data, test_data, 
                                     baseline_feature_ranking=baseline_ranking)
    
    # Se for a Linha de Base, capturar o ranking para uso futuro
    if is_baseline:
        baseline_ranking = current_results['Feature_Importance_Ranking']
        print("-> Linha de Base estabelecida.")

    # Adicionar metadados ao resultado
    current_results['Condition'] = 'Baseline' if is_baseline else dir_path.split('/')[-1].split('_')[0]
    current_results['Parameter_Value'] = 'N/A' if is_baseline else dir_path.split('_')[-1]
    
    # Converter para DataFrame de uma linha e anexar ao mestre
    results_row = pd.DataFrame([current_results])
    master_results_df = pd.concat([master_results_df, results_row], ignore_index=True)
    
    print(f"   -> EOD: {current_results['Equal_Opportunity_Difference']:.3f} | F1: {current_results['Overall_F1_Score']:.3f} | Estabilidade SHAP: {current_results['SHAP_Rank_Stability']:.3f}")

# Remover o ranking SHAP complexo para simplificar o arquivo final
master_results_df = master_results_df.drop(columns=['Feature_Importance_Ranking'], errors='ignore')

# Salvar a tabela final
MASTER_RESULTS_FILE = 'project_audit_results.csv'
master_results_df.to_csv(MASTER_RESULTS_FILE, index=False)
print("\n--- Loop Concluído ---")
print(f"Resultados salvos em: {MASTER_RESULTS_FILE}")